<a href="https://colab.research.google.com/github/hamletbatista/techseoboost/blob/master/GenerateTextFromTweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Here is the plan:

We are going to pull the most recent tweets from the top Twitter users and use those tweets to generate new original content.

1. We will pull tweets automatically
2. We will clean up the tweets
3. We will generate 256-word articles from the tweets
4. We will review the content generated for factual accuracies and errors
5. We will measure the Real/Fake ratio



#Pull tweets automatically and pickle results

We are going to get the top most followed Twitter users from this list (including only peoeple, not organizations) https://www.brandwatch.com/blog/most-twitter-followers/

BarackObama



In [1]:
!pip install tweepy

In [0]:
config_file="""
consumer_key: <CHANGEME>
consumer_secret:  <CHANGEME>
access_key:  <CHANGEME>
access_secret:  <CHANGEME>

twitter_users:
  - BarackObama
  - katyperry
  - justinbieber
  - rihanna
  - taylorswift13
  - Cristiano
  - ladygaga
  - TheEllenShow
  - ArianaGrande
  - realDonaldTrump
  - jtimberlake
  - KimKardashian
  - selenagomez
  - britneyspears
  - shakira
  - jimmyfallon
  - narendramodi

"""

with open("config.yml", "w") as f:
  f.write(config_file)

In [0]:
#%cd ..

In [0]:
#!python3 tweet_generator.py
import yaml
import tweepy
import re
import pickle


In [100]:

with open("config.yml", "r") as f:
    cfg = yaml.load(f)

auth = tweepy.OAuthHandler(cfg['consumer_key'], cfg['consumer_secret'])
auth.set_access_token(cfg['access_key'], cfg['access_secret'])

api = tweepy.API(auth)

for user in cfg['twitter_users']:
    #fixed bug: texts was not resetting per user
    texts = []
    print("Downloading {}'s Tweets...".format(user))
    all_tweets = tweepy.Cursor(api.user_timeline,
                               screen_name=user,
                               count=200,
                               tweet_mode='extended',
                               include_rts=False).pages(16)

    for page in all_tweets:
      for tweet in page:
          if tweet.full_text is not '':
              texts.append(tweet.full_text)
              
    with open(user+"_tweets.pkl", "wb") as fp:   #Pickling
      print("Saving {}'s Tweets ...".format(user))
      pickle.dump(texts, fp)


Saving BarackObama's Tweets ...
Saving katyperry's Tweets ...
Saving justinbieber's Tweets ...
Saving rihanna's Tweets ...
Saving taylorswift13's Tweets ...
Saving Cristiano's Tweets ...
Saving ladygaga's Tweets ...
Saving TheEllenShow's Tweets ...
Saving ArianaGrande's Tweets ...
Saving realDonaldTrump's Tweets ...
Saving jtimberlake's Tweets ...
Saving KimKardashian's Tweets ...
Saving selenagomez's Tweets ...
Saving britneyspears's Tweets ...
Saving shakira's Tweets ...
Saving jimmyfallon's Tweets ...
Saving narendramodi's Tweets ...


In [102]:
#!rm tweets.zip
!zip tweets.zip *pkl

  adding: ArianaGrande_tweets.pkl (deflated 58%)
  adding: BarackObama_tweets.pkl (deflated 64%)
  adding: britneyspears_tweets.pkl (deflated 58%)
  adding: Cristiano_tweets.pkl (deflated 59%)
  adding: jimmyfallon_tweets.pkl (deflated 60%)
  adding: jtimberlake_tweets.pkl (deflated 56%)
  adding: justinbieber_tweets.pkl (deflated 59%)
  adding: katyperry_tweets.pkl (deflated 54%)
  adding: KimKardashian_tweets.pkl (deflated 60%)
  adding: ladygaga_tweets.pkl (deflated 55%)
  adding: narendramodi_tweets.pkl (deflated 62%)
  adding: realDonaldTrump_tweets.pkl (deflated 60%)
  adding: rihanna_tweets.pkl (deflated 55%)
  adding: selenagomez_tweets.pkl (deflated 57%)
  adding: shakira_tweets.pkl (deflated 60%)
  adding: taylorswift13_tweets.pkl (deflated 54%)
  adding: TheEllenShow_tweets.pkl (deflated 58%)


In [0]:
from google.colab import files



In [0]:
#!ls
files.download("tweets.zip")
#files.upload()

In [16]:
#!unzip tweets.zip

Archive:  tweets.zip
  inflating: ArianaGrande_tweets.pkl  
  inflating: BarackObama_tweets.pkl  
  inflating: britneyspears_tweets.pkl  
  inflating: Cristiano_tweets.pkl    
  inflating: jimmyfallon_tweets.pkl  
  inflating: jtimberlake_tweets.pkl  
  inflating: justinbieber_tweets.pkl  
  inflating: katyperry_tweets.pkl    
  inflating: KimKardashian_tweets.pkl  
  inflating: ladygaga_tweets.pkl     
  inflating: narendramodi_tweets.pkl  
  inflating: realDonaldTrump_tweets.pkl  
  inflating: rihanna_tweets.pkl      
  inflating: selenagomez_tweets.pkl  
  inflating: shakira_tweets.pkl      
  inflating: taylorswift13_tweets.pkl  
  inflating: TheEllenShow_tweets.pkl  


In [0]:
def process_tweet_text(text):
    text = re.sub(r'http\S+', '', text)   # Remove URLs
    text = re.sub(r'@[a-zA-Z0-9_]+', '', text)  # Remove @ mentions
    text = re.sub(r'#[^\s]+', '', text)  # Remove # hashtags
    text = text.strip(" ")   # Remove whitespace resulting from above
    text = re.sub(r' +', ' ', text)   # Remove redundant spaces

    # Handle common HTML entities
    text = re.sub(r'&lt;', '<', text)
    text = re.sub(r'&gt;', '>', text)
    text = re.sub(r'&amp;', '&', text)
    return text



In [0]:
def load_tweets(user_file):
  texts = []
  #with open(user+"_tweets.pkl", "rb") as fp:   #Pickling
  with open(user_file, "rb") as fp:   #Pickling
    print("Reading {}'s Tweets ...".format(name_from_file(user_file)))
    texts = pickle.load(fp)

    return texts

In [128]:
!ls

ArianaGrande_tweets.pkl   jtimberlake_tweets.pkl      seqlen256_v1.ckpt
BarackObama_tweets.pkl	  justinbieber_tweets.pkl     shakira_tweets.pkl
britneyspears_tweets.pkl  katyperry_tweets.pkl	      source_attribution.py
CODE_OF_CONDUCT.md	  KimKardashian_tweets.pkl    taylorswift13_tweets.pkl
codes			  ladygaga_tweets.pkl	      TheEllenShow_tweets.pkl
config.yml		  LICENSE.txt		      transformer.py
control_codes.txt	  narendramodi_tweets.pkl     transformer.pyc
Cristiano_tweets.pkl	  README.md		      tweets.zip
estimator.patch		  realDonaldTrump_tweets.pkl  vocab
generation.py		  rihanna_tweets.pkl
jimmyfallon_tweets.pkl	  selenagomez_tweets.pkl


In [107]:
#texts = load_tweets("ArianaGrande_tweets.pkl")
#!rm *pkl

Reading ArianaGrande's Tweets ...


In [108]:
#texts[:10]

['yeeee https://t.co/IPlsgnaSWO',
 '👼🏼 https://t.co/B3vPrN69GK https://t.co/NOdAQTDcUO',
 'cherishing these last three shows in europe because ᵗʰᵉ ˢʰᵒʷ’ˢ ᵍᵒⁿⁿᵃ ᵇᵉ ˢˡᶦᵍʰᵗˡʸ ᵈᶦᶠᶠᵉʳᵉⁿᵗ ⁿᵉˣᵗ ˡᵉᵍ',
 'you’re sooooooooooo fuckin funny man. and cute. i love our lil moments we share during the show (well, those of u i can see w my trash eyes)',
 'love u w all my heart. three more, europe. although we’re all v tired and ready for a lil break i’m already...... the most emotional.',
 'i don’t think y’all understand how much i love and value our relationship. ugh. this show. you. all of it.',
 'stayin up all night, order me pad thai, then we gon sleeep til ⁿᵒ⁰ⁿ https://t.co/rK9H2O3atd',
 '@diornasa @angeIfalls @grandetension @ghostindeIuxe @joeljoab_ love u. we takin off! miss and love y’all always. thank u for everything. never a min i’m not grateful for u.',
 '@angeIfalls @grandetension @ghostindeIuxe @joeljoab_ me too ! thank y’all. so much. a year ago i never thought i’d be here rn. for sure fo

In [22]:
#texts[0]

'Here’s an interesting read—a reminder that behind every opinion lies a human being with real experiences and a story to tell. Sometimes we’ll agree and sometimes we won’t, but if we want our democracy work, listening to each other isn’t optional. https://t.co/8d4fPUprlu'

In [0]:
#import re, requests

#short_links = re.findall("https?://t.co/\S+", texts[0])
#short_links

In [0]:
#r = requests.get(short_links[0])
#r.status_code

In [0]:
#r.url

In [0]:
import requests, re 

def expand_short_links(text):
  #urls = list()

  new_text = text

  short_links = re.findall("https?://t.co/\S+", texts[0])
  for short_link in short_links:
    r = requests.get(short_link)

    if r.status_code == 200:
      #urls.append(r.url)

      new_text = new_text.replace(short_link, r.url)
    else:
      print("URL {url} failed with status code {code}".format(url=r.url, code=r.status_code))

  

  return new_text



In [27]:
expand_short_links(texts[0])

'Here’s an interesting read—a reminder that behind every opinion lies a human being with real experiences and a story to tell. Sometimes we’ll agree and sometimes we won’t, but if we want our democracy work, listening to each other isn’t optional. https://www.nytimes.com/interactive/2019/10/02/upshot/these-526-voters-represent-america.html'

In [0]:
!ls

config.yml  oprah_tweets.pkl  sample_data  tweet-generator


#CTRL code to generate texts

1. Take the last 5 Tweets from each celebrity and generate 3 256-word articles
2. Try different tweet permutations: like it is, with URL first, URL only


In [4]:
!nvidia-smi

Sun Oct 13 02:10:59 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [5]:
!git clone https://github.com/salesforce/ctrl

Cloning into 'ctrl'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 123 (delta 72), reused 51 (delta 22), pack-reused 0
Receiving objects: 100% (123/123), 24.25 MiB | 19.07 MiB/s, done.
Resolving deltas: 100% (72/72), done.


In [6]:
%cd ctrl/

/content/ctrl


In [7]:
!git checkout lower_memory

Branch 'lower_memory' set up to track remote branch 'lower_memory' from 'origin'.
Switched to a new branch 'lower_memory'


In [8]:
!pip2 install tensorflow-gpu==1.14

     |████████████████████████████████| 377.0MB 68kB/s 
     |████████████████████████████████| 3.2MB 43.6MB/s 
     |████████████████████████████████| 491kB 46.8MB/s 
ERROR: tensorflow 1.15.0rc3 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.14.0 which is incompatible.
ERROR: tensorflow 1.15.0rc3 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 1.14.0 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.0
    Uninstalling tensorflow-estimator-1.15.0:
      Successfully uninstalled tensorflow-estimator-1.15.0


In [9]:
!patch -b /usr/local/lib/python2.7/dist-packages/tensorflow_estimator/python/estimator/keras.py estimator.patch

patching file /usr/local/lib/python2.7/dist-packages/tensorflow_estimator/python/estimator/keras.py


In [10]:
!pip2 install fastBPE

  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp27-cp27mu-linux_x86_64.whl size=472510 sha256=6b1969d98e89be4709cf0fe43c0e8bd0736d954f9a0e8993098753eb3dd035a2
  Stored in directory: /root/.cache/pip/wheels/f3/0c/9c/fc62058b4d473a5602bcd3d3edfece796f123875379ea82d79
Successfully built fastBPE


In [0]:
#!pip2 install gsutil

In [11]:
!gsutil -m cp -r gs://sf-ctrl/seqlen256_v1.ckpt .

Copying gs://sf-ctrl/seqlen256_v1.ckpt/checkpoint...
Copying gs://sf-ctrl/seqlen256_v1.ckpt/model.ckpt-413000.data-00000-of-00001...
Copying gs://sf-ctrl/seqlen256_v1.ckpt/model.ckpt-413000.index...
Copying gs://sf-ctrl/seqlen256_v1.ckpt/model.ckpt-413000.meta...
\ [4/4 files][ 12.7 GiB/ 12.7 GiB] 100% Done  44.7 MiB/s ETA 00:00:00           
Operation completed over 4 objects/12.7 GiB.                                     


In [14]:
!ls

CODE_OF_CONDUCT.md  estimator.patch  README.md		    transformer.py
codes		    generation.py    seqlen256_v1.ckpt	    vocab
control_codes.txt   LICENSE.txt      source_attribution.py


In [0]:
#!python2 generation.py --model seqlen256_v1.ckpt/model.ckpt-413000.data-00000-of-00001 --print_once

In [0]:
#s=""" 
#This is a “Test”, please ‘don’t worry too much’. It will –happen soon, Don’t dismay.
#"""

In [0]:
#for c in s:
#  if ord(c) >= 128:
#    print(c, ord(c))

In [0]:
tweet = """https://www.nytimes.com/interactive/2019/10/02/upshot/these-526-voters-represent-america.html

Here’s an interesting read—a reminder that behind every opinion lies a human being with real experiences and a story to tell. Sometimes we’ll agree and sometimes we won’t, but if we want our democracy work, listening to each other isn’t optional."""

In [60]:
#new_text = replace_unicode(tweet)

’ 8217
— 8212
’ 8217
’ 8217
’ 8217


In [62]:
#print(new_text)

https://www.nytimes.com/interactive/2019/10/02/upshot/these-526-voters-represent-america.html

Here's an interesting read-a reminder that behind every opinion lies a human being with real experiences and a story to tell. Sometimes we'll agree and sometimes we won't, but if we want our democracy work, listening to each other isn't optional.


In [64]:
#!python2 generation.py --model seqlen256_v1.ckpt/model.ckpt-413000.data-00000-of-00001 --print_once

W1012 17:10:55.670391 140574309554048 deprecation_wrapper.py:119] From generation.py:6: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.

W1012 17:10:55.689090 140574309554048 deprecation_wrapper.py:119] From generation.py:43: The name tf.random.set_random_seed is deprecated. Please use tf.compat.v1.random.set_random_seed instead.

246534 unique words
2019-10-12 17:10:56.010863: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2019-10-12 17:10:56.034858: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-10-12 17:10:56.035814: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-10-12 17:10:56.040

#Write the last 3 tweet per celebirty to a JSON file. 
Make sure to replace unicode chars

In [0]:
#TODO add more exceptions
replacements = {
                237: "i",
                225: "a",
                201: "E",
                227: "a",
                243: "o",
                237: "i",
                8230: "...",
                8220: '"',
                8221: '"',
                8216: "'",
                8217: "'",
                8211: "-",
                8212: "-",
                }

In [0]:
def replace_unicode(text):
  """
    Check if there are unicode characters and replace them for ascii equivalent per the replacement table
  """

  new_text = ""

  for c in text:
    if ord(c) >= 128:

      print(c, ord(c))
      
      if ord(c) in [8211, 8212, 8216, 8217, 8220, 8221]:
        new_text = new_text + replacements[ord(c)]

      else:
        new_text = new_text + " " #replace anything else for a space

    else:
      new_text = new_text + c
  return new_text


In [65]:
!ls

ArianaGrande_tweets.pkl   jtimberlake_tweets.pkl      selenagomez_tweets.pkl
BarackObama_tweets.pkl	  justinbieber_tweets.pkl     seqlen256_v1.ckpt
britneyspears_tweets.pkl  katyperry_tweets.pkl	      shakira_tweets.pkl
CODE_OF_CONDUCT.md	  KimKardashian_tweets.pkl    source_attribution.py
codes			  ladygaga_tweets.pkl	      taylorswift13_tweets.pkl
control_codes.txt	  LICENSE.txt		      TheEllenShow_tweets.pkl
Cristiano_tweets.pkl	  narendramodi_tweets.pkl     transformer.py
estimator.patch		  README.md		      transformer.pyc
generation.py		  realDonaldTrump_tweets.pkl  tweets.zip
jimmyfallon_tweets.pkl	  rihanna_tweets.pkl	      vocab


In [0]:
def name_from_file(tweet_file):
  return tweet_file.replace("_tweets.pkl", "")

In [0]:
def remove_short_links(text):
  #urls = list()

  new_text = text

  short_links = re.findall("https?://t.co/\S+", text)
  for short_link in short_links:
    new_text = new_text.replace(short_link, "")

  return new_text


In [0]:
from glob import glob
from collections import defaultdict

def tweets_to_dict(pattern="*pkl", last=3):

  tweet_files = glob(pattern)

  tweet_texts = defaultdict(list)

  for tweet_file in tweet_files:

    user = name_from_file(tweet_file)
    print(tweet_file)
    tweets = load_tweets(tweet_file)

    #print(len(tweets))
    #print(tweets[:3])

    for tweet in tweets:#[:last]:

      print(tweet)
      clean_tweet = replace_unicode(tweet)
      clean_tweet = remove_short_links(clean_tweet)

      #print(clean_tweet)
      if len(clean_tweet) > 50:
        tweet_texts[user].append(clean_tweet)

        #we need only 3 good tweets
        if len(tweet_texts[user]) >= last:
          break
    
  return tweet_texts
  






In [145]:
#tweet_texts = tweets_to_dict()

shakira_tweets.pkl
Reading shakira's Tweets ...
EXCLUSIVA! No se pierdan este espectacular video de La La La / Waka Waka del #ShakiraFilm (en cines el 13 de Noviembre). Aquí https://t.co/WxEI2AB8IX encontrarán el video entero! ShakHQ https://t.co/uACctBW9ZH
í 237
á 225
EXCLUSIVE! Watch Shak's stunning performance of La La La / Waka Waka from #ShakiraFilm (in cinemas worldwide, Nov 13). See the full video at https://t.co/WxEI2AB8IX ShakHQ https://t.co/tXurorgudS
Miren con quien estoy!!!!! https://t.co/z5aLuiSWaw
And the finale for @MessiCirque’s, “lalala” a pleasant surprise! @Cirque @TeamMessi https://t.co/n8NOnQTqLT
’ 8217
“ 8220
” 8221
ArianaGrande_tweets.pkl
Reading ArianaGrande's Tweets ...
yeeee https://t.co/IPlsgnaSWO
👼🏼 https://t.co/B3vPrN69GK https://t.co/NOdAQTDcUO
👼 128124
🏼 127996
cherishing these last three shows in europe because ᵗʰᵉ ˢʰᵒʷ’ˢ ᵍᵒⁿⁿᵃ ᵇᵉ ˢˡᶦᵍʰᵗˡʸ ᵈᶦᶠᶠᵉʳᵉⁿᵗ ⁿᵉˣᵗ ˡᵉᵍ
ᵗ 7511
ʰ 688
ᵉ 7497
ˢ 738
ʰ 688
ᵒ 7506
ʷ 695
’ 8217
ˢ 738
ᵍ 7501
ᵒ 7506
ⁿ 8319
ⁿ 8319
ᵃ 7491
ᵇ 74

In [0]:
#replace_unicode(tweet_texts["ArianaGrande"][0])

In [144]:
#remove_short_links(tweet_texts["KimKardashian"][0])

'Haley wears the @skims Triangle Bralette ($32) in Mineral, coming soon in sizes XXS - 4X. Shop the Cotton Collection on October 15 at 9AM PST / 12PM EST and join the waitlist now to be the first to shop.\nPhoto: #VanessaBeecroft '

In [45]:
tweet_texts

defaultdict(list,
            {'ArianaGrande': ["cherishing these last three shows in europe because         '                                      ",
              "you're sooooooooooo fuckin funny man. and cute. i love our lil moments we share during the show (well, those of u i can see w my trash eyes)",
              "love u w all my heart. three more, europe. although we're all v tired and ready for a lil break i'm already...... the most emotional."],
             'BarackObama': ["Congrats to the @WashMystics on a gutsy, first-ever championship! A great team performance when it counted. If folks aren't careful, this title thing might become a habit in DC.",
              "Here's an interesting read-a reminder that behind every opinion lies a human being with real experiences and a story to tell. Sometimes we'll agree and sometimes we won't, but if we want our democracy work, listening to each other isn't optional. ",
              "As President, I leaned on @AmbassadorRice's exper

In [0]:
with open("celeb_tweets.pkl", "wb") as f:
  pickle.dump(tweet_texts, f, protocol=2) #to support python 2

In [0]:
#%pycat generation.py

In [18]:
#files.download("celeb_tweets.pkl")
files.upload()

Saving celeb_tweets (1).pkl to celeb_tweets (1).pkl


{'celeb_tweets (1).pkl': b'\x80\x02ccollections\ndefaultdict\nq\x00c__builtin__\nlist\nq\x01\x85q\x02Rq\x03(X\x07\x00\x00\x00shakiraq\x04]q\x05(X\xa3\x00\x00\x00EXCLUSIVA! No se pierdan este espectacular video de La La La / Waka Waka del #ShakiraFilm (en cines el 13 de Noviembre). Aqu   encontrar n el video entero! ShakHQ q\x06X\x95\x00\x00\x00EXCLUSIVE! Watch Shak\'s stunning performance of La La La / Waka Waka from #ShakiraFilm (in cinemas worldwide, Nov 13). See the full video at  ShakHQ q\x07XT\x00\x00\x00And the finale for @MessiCirque\'s, "lalala" a pleasant surprise! @Cirque @TeamMessi q\x08eX\x0c\x00\x00\x00ArianaGrandeq\t]q\n(Xc\x00\x00\x00cherishing these last three shows in europe because         \'                                      q\x0bX\x8c\x00\x00\x00you\'re sooooooooooo fuckin funny man. and cute. i love our lil moments we share during the show (well, those of u i can see w my trash eyes)q\x0cX\x85\x00\x00\x00love u w all my heart. three more, europe. although we\'re

In [0]:
!mv 'celeb_tweets (1).pkl' celeb_tweets.pkl

In [0]:
import pickle

with open("celeb_tweets.pkl", "rb") as f:
  tweet_texts = pickle.load(f)

In [0]:
import re
def get_usernames(text):
  #urls = list()

  #new_text = text

  usernames_dict = dict()

  user_names = re.findall("@[^\s',\.]+", text)
  for user_name in user_names:
    #new_text = new_text.replace(short_link, "")
    print(user_name[1:])
    usernames_dict[user_name[1:]] = ""

  return usernames_dict


In [40]:
usernames = dict()
for celeb in tweet_texts.keys():
  for tweet in tweet_texts[celeb]:
    #print("Links {celeb} says \"{tweet}\"".format(celeb=celeb,tweet=tweet))
    usernames.update(get_usernames(tweet))

@MessiCirque
@Cirque
@TeamMessi
@skims
@PilgrimageFest
@voguemagazine
@FentyOfficial
@ethjgreen
@yusefhairnyc
@phaidon
@fentybeauty
@BNUnionSquareNY
@seanseaevans
@MonsterEnergy
@NASCAR
@vanswarpedtour
@AmericanIdol
@LukeBryanOnline
@lionelrichie
@face_lace
@hauslabs
@ChuckGrassley
@joniernst
@debfisher
@BenSasse
@WEmovement
@ABCNetwork
@DanAndShay
@gummymoonlight
@Dev_Fadnavis
@WashMystics
@AmbassadorRice
@NPR
@BobBoilen
@nbcsnl
@TheKalenAllen
@official_tWitch
@NickCannon
@NicoleScherzy
@official_tWitch
@TheKalenAllen


In [0]:
usernames = {'ABCNetwork': 'ABC',
 'AmbassadorRice': 'Susan Rice',
 'AmericanIdol': 'American Idol',
 'BNUnionSquareNY': 'Barnes & Noble Union Square',
 'BenSasse': 'Ben Sasse',
 'BobBoilen': 'Bob Boilen',
 'ChuckGrassley': 'Chuck Grassley',
 'Cirque': 'Cirque du Soleil',
 'DanAndShay': 'Dan + Shay',
 'Dev_Fadnavis': 'Devendra Fadnavis',
 'FentyOfficial': 'Fenty',
 'LukeBryanOnline': 'Luke Bryan',
 'MessiCirque': 'Messi by Cirque du Soleil',
 'MonsterEnergy': 'Monster Energy',
 'NASCAR': 'NASCAR',
 'NPR': 'NPR',
 'NickCannon': 'Nick Cannon',
 'NicoleScherzy': 'Nicole Scherzy',
 'PilgrimageFest': 'Pilgrimage Festival',
 'TeamMessi': 'Team Messi',
 'TheKalenAllen': 'Kalen Allen',
 'WEmovement': 'WE',
 'WashMystics': 'Washington Mystics',
 'debfisher': 'Deborah Fisher',
 'ethjgreen': 'Ethan James Green',
 'face_lace': 'Face Lace',
 'fentybeauty': 'Fenty Beauty',
 'gummymoonlight': 'hanna',
 'hauslabs': 'HAUS LABORATORIES',
 'joniernst': 'Joni Ernst',
 'lionelrichie': 'Lionel Richie',
 'nbcsnl': 'Saturday Night Live',
 'official_tWitch': 'Stephen tWitch Boss',
 'phaidon': 'Phaidon',
 'seanseaevans': 'Sean Evans',
 'skims': 'SKIMS',
 'vanswarpedtour': 'Vans Warped Tour',
 'voguemagazine': 'Vogue Magazine',
 'yusefhairnyc': 'Yusef Williams'}

In [0]:
def replace_usernames(text):
  #urls = list()

  new_text = text

  #usernames_dict = dict()

  user_names = re.findall("@[^\s',\.]+", text)
  for user_name in user_names:
    new_text = new_text.replace(user_name, usernames[user_name[1:]])
    #print(user_name[1:])
    #usernames_dict[user_name[1:]] = ""

  return new_text

In [0]:
user2name = {
      "BarackObama" : "Barack Obama",
      "katyperry" : "Katy Perry",
      "justinbieber" : "Justin Bieber",
      "rihanna" : "Rihanna",
      "taylorswift13" : "Tyalor Swift",
      "Cristiano" : "Cristiano Ronaldo",
      "ladygaga" : "LAdy Gaga",
      "TheEllenShow": "Ellen DeGeneres",
      "ArianaGrande": "Ariana Grande",
      "realDonaldTrump" : "Donald Trump",
      "jtimberlake" : "Justin Timberlake",
      "KimKardashian" : "Kim Kardashian",
      "selenagomez" : "Selena Gomez",
      "britneyspears" : "Britney Spears",
      "shakira" : "Shakira",
      "jimmyfallon" : "Jimmy Fallon",
      "narendramodi": "Narend Ramodi"
}

In [0]:
#usernames = dict()
prompts = list()

for celeb in tweet_texts.keys():
  for tweet in tweet_texts[celeb]:

    #remove hashtag
    tweet = tweet.replace("#", "")
    prompts.append("Links {celeb} says \"{tweet}\"".format(celeb=user2name[celeb], tweet=replace_usernames(tweet)))
    #usernames.update(get_usernames(tweet))

In [52]:
prompts

['Links Shakira says "EXCLUSIVA! No se pierdan este espectacular video de La La La / Waka Waka del ShakiraFilm (en cines el 13 de Noviembre). Aqu   encontrar n el video entero! ShakHQ "',
 'Links Shakira says "EXCLUSIVE! Watch Shak\'s stunning performance of La La La / Waka Waka from ShakiraFilm (in cinemas worldwide, Nov 13). See the full video at  ShakHQ "',
 'Links Shakira says "And the finale for Messi by Cirque du Soleil\'s, "lalala" a pleasant surprise! Cirque du Soleil Team Messi "',
 'Links Ariana Grande says "cherishing these last three shows in europe because         \'                                      "',
 'Links Ariana Grande says "you\'re sooooooooooo fuckin funny man. and cute. i love our lil moments we share during the show (well, those of u i can see w my trash eyes)"',
 'Links Ariana Grande says "love u w all my heart. three more, europe. although we\'re all v tired and ready for a lil break i\'m already...... the most emotional."',
 'Links Kim Kardashian says "Hal

In [0]:
with open("prompts.pkl", "wb") as f:
  pickle.dump(prompts, f, protocol=2)

In [73]:
%%writefile generate2.py

from __future__ import division
from __future__ import print_function
import tensorflow as tf
import os
import numpy as np
tf.enable_eager_execution()
import transformer
import argparse
import pdb
import sys
import re
from collections import Counter
from tensorflow.python import debug as tf_debug
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import embedding_ops
from tensorflow.python import pywrap_tensorflow
import fastBPE
import platform

use_py3 = platform.python_version()[0] == '3'

parser = argparse.ArgumentParser(description='TensorFlow code for generating from CTRL')
parser.add_argument('--model_path', type=str, required=True,
                                        help='location of model *data* checkpoint; this is NOT the directory but rather the model checkpoint')
parser.add_argument('--seed', type=int, default=1337,
                                        help='random seed for TensorFlow, numpy and PythonHash')
parser.add_argument('--generate_num', type=int, default=256,
                                        help='number of tokens to generate')
parser.add_argument('--temperature', type=float, default=0,
                                        help='temperature for sampling distribution; 0 means greedy')
parser.add_argument('--nucleus', type=float, default=0.,
                                        help='cumulative probability cutoff for nucleus sampling; 0 means no nucleus sampling')
parser.add_argument('--topk', type=int, default=0,
                                        help='topk value for sampling from the softmax distribution ; 0 means no topk preferred')
parser.add_argument('--penalty', type=float, default=1.2,
                                        help='repetition penalty for greedy sampling')
parser.add_argument('--print_once', action='store_true',
                                        help='the completion is printed only at the end; not every word')
parser.add_argument('--topn', type=int, default=0,
                                        help='print top-n candidates during generations; defaults to 0 which is no printing')

args = parser.parse_args()
tf.random.set_random_seed(args.seed)
os.environ['PYTHONHASHSEED'] = str(args.seed)
np.random.seed(args.seed)

# load the vocabulary from file
vocab = open('vocab').read().decode(encoding='utf-8').split('\n') if not use_py3 else open('vocab', encoding='utf-8').read().split('\n')
vocab = list(map(lambda x: x.split(' ')[0], vocab)) + ['<unk>'] + ['\n']
print ('{} unique words'.format(len(vocab)))

# length of the vocabulary
vocab_size = len(vocab)

# define the numericalization map
# idx2word maps the numericalized ID to the word
# word2idx maps the word to the numericalized ID
word2idx = {u:i for i, u in enumerate(vocab)}
idx2word = np.array(vocab)



# sequence length to use for the transformer
# the model is trained with a seq_length of 512
# so, any value <= 512 should work
seq_length = min(args.generate_num, 256)




# the dimension of the transformer
embedding_dim = 1280


# Now, we begin defining the model
# we defer the transformer definition to transformer.py
# here, we only define the tied softmax layer
# this layer ties the softmax weights to the input embeddings
class TiedEmbeddingSoftmax(tf.keras.layers.Layer):

  def __init__(self, vocab_size=vocab_size, embedding_size=embedding_dim, **kwargs):
    super(TiedEmbeddingSoftmax, self).__init__()
    self.w = self.add_weight(name='w', shape=(vocab_size, embedding_size), dtype=tf.float32, 
                             initializer='random_normal',
                             trainable=True)
    self.b = self.add_weight(name='b', shape=(vocab_size,), dtype=tf.float32, 
                             initializer='zeros',
                             trainable=True)

  def call(self, inputs, embed=True):
    if embed:
      dtype = tf.keras.backend.dtype(inputs)
      if dtype != 'int32' and dtype != 'int64':
        inputs = math_ops.cast(inputs, 'int32')
      return embedding_ops.embedding_lookup(self.w, inputs)
    else:
      return tf.tensordot(inputs, tf.transpose(self.w), 1) + self.b

# input for the keras model
tokens = tf.keras.layers.Input(shape=(seq_length,), dtype='int32')

# instantiates a tied softmax class
tied_embedding_softmax = TiedEmbeddingSoftmax()

# embedded tokens, before passing it to the transformer
embedded = tied_embedding_softmax(tokens, embed=True)

# the activations after passing it from the transformer
# for some odd reason, TPUs don't play well with specifying the arguments of the Encoder() function
# so you have to leave them at their defaults
transformed = transformer.Encoder()(embedded, training=False)


# pass the activations from our tiedsoftmax class
# this time with embed=False denoting that we are doing the softmax operation
# and not a lookup
logits = tied_embedding_softmax(transformed, embed=False)


# finally, define the Keras model with inputs as tokens and outputs as the logits we just computed
model = tf.keras.Model(inputs=tokens, outputs=logits)


# the loss function is a simple categorical crossentropy between the logits and the labels
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# the optimizer is not used since this code only supports inference
# however, to compile the model, we still define it
optimizer = tf.contrib.tpu.CrossShardOptimizer(
    tf.contrib.estimator.clip_gradients_by_norm(
        tf.train.AdagradOptimizer(learning_rate=1e-2), 0.25)
    )        

# compile the model with the optimizer and loss            
model.compile(optimizer=optimizer, loss=loss)
print(model.summary())


# almost there, we now take the user prompt and tokenize with BPE
# load BPE codes
bpe = fastBPE.fastBPE('codes', 'vocab')

temperature = args.temperature
nucleusprob = args.nucleus
penalty = args.penalty
topk = args.topk

# Load the model file
chkpt_for_reader = '.'.join(args.model_path.split('.')[:-1])
reader = pywrap_tensorflow.NewCheckpointReader(chkpt_for_reader)

# assign weights from the checkpoint to the Keras model
# this is super hacky but I couldn't find a better way to do this
# PR is highly welcome if you know of a better way

# embedding and softmax
# these are fp32
model.layers[1].trainable_variables[0].assign(tf.cast(reader.get_tensor('w'), tf.float32))
model.layers[1].trainable_variables[1].assign(tf.cast(reader.get_tensor('b'), tf.float32))

# encoder weights
for _ in range(len(model.layers[2].trainable_weights)):
    tensor = model.layers[2].trainable_weights[_]
    if 'normalization' in tensor.name[:-2]: # layernorm is fp32
      tensor.assign(tf.cast(reader.get_tensor(tensor.name[:-2]), tf.float32))
    else: # everything else is fp16
      tensor.assign(tf.cast(reader.get_tensor(tensor.name[:-2]), tf.float16))

#testing
import pickle

#with open("celeb_tweets.pkl", "rb") as f:
#  tweet_texts = pickle.load(f)

with open("prompts.pkl", "rb") as f:
  prompts = pickle.load(f)

#for celeb in tweet_texts.keys():
#  for tweet in tweet_texts[celeb]:
    #print("Links {celeb} says \"{tweet}\"".format(celeb=celeb,tweet=tweet))
#while True:
file_number = 0

for prompt in prompts:
    #print('WARNING! THIS VERSION OF THE CODE ALLOWS FOR LOWER MEMORY USAGE THROUGH FP16 QUANTIZATION BUT IS UNTESTED; GENERATIONS MAY BE WORSE. USE AT YOUR OWN RISK. ')
    #prompt = "Links {celeb} says \"{tweet}\"".format(celeb=celeb,tweet=tweet) #raw_input('ENTER PROMPT: ') if not use_py3 else input('ENTER PROMPT: ')

    #resume from last run
    if file_number <= 5:
      print("Skipping prompt: {prompt}".format(prompt=prompt))
      file_number = file_number + 1
      continue

    # tokenize provided prompt
    split_prompt = bpe.apply([prompt])[0].split()

    try:
      text = [word2idx[i] for i in split_prompt]
    except KeyError:
      print("Skipping prompt: {prompt}".format(prompt=prompt))
      file_number = file_number + 1
      continue

    # pad with 0s and create a mini-batch of 2 (arbitrary, for ease of code)
    padded_text = text + [0] * (args.generate_num - len(text))
    tokens_generated = np.tile(padded_text, (1,1))
    try:
        for token in range(len(text)-1, args.generate_num-1):
          # get the logits from the prediction function
          # the logic here is a bit convoluted because we are allowing generation past 512 tokens
          # this is done by sliding the window over (past 512 tokens) and continuing prediction
          # I'm sure this can be simplified (TODO)
          if token <= seq_length:
            prompt_logits = model.predict_on_batch(tokens_generated[:, :seq_length]).squeeze() / (temperature if temperature>0 else 1.)
            _token = token if token < seq_length else -1
          else:
            _token = -1
            end = token + 1
            start = token - seq_length + 2
            prompt_logits = model.predict_on_batch(np.hstack((tokens_generated[:,0:1], tokens_generated[:,start:end]))).squeeze() / (temperature if temperature>0 else 1.)


          # if penalty (for repetition) is non-zero,
          # discount the logits from already generated tokens
          if penalty>0:
              penalized_so_far = set()
              for _ in range(token+1):
                 generated_token = tokens_generated[0][_]
                 # don't penalize newlines
                 # you could also choose not to penalize frequent words
                 # (which incidentally are sorted in the vocab file)
                 # but I don't do that
                 # if it prints too many new lines instead of continuing generating text,
                 # you might want to comment this out
                 if idx2word[generated_token] == '\n':
                     continue
                 if generated_token in penalized_so_far:
                     continue
                 penalized_so_far.add(generated_token)
                 prompt_logits[_token][generated_token] /= penalty

          # disallow some tokens
          prompt_logits[_token][word2idx['<unk>']] = -1e8

          # sometimes, when generating from reddit,
          # it tries to generate the Score (reddit Karma) immediately after generating the Title:
          # to disallow this, we can just prevent it from generating Score
          prompt_logits[_token][word2idx['Sco@@']] = -1e8


          # compute probabilities from logits
          prompt_probs = np.exp(prompt_logits[_token])
          prompt_probs = prompt_probs / sum(prompt_probs)
          pruned_list = np.argsort(prompt_probs)[::-1]
          # if you are using nucleus prob, then compute the nucleus probability size
          if nucleusprob > 0.:
            minimum_topk = 1
            nucleus = max(np.where(np.cumsum(np.sort(prompt_probs)[::-1])>nucleusprob)[0][0], minimum_topk)
          elif topk > 0:
            # we are over-loading notation here
            # if you choose to specify a topk instead of a nucleus,
            # we will hardcode the nucleus to be just that
            nucleus = topk
          else:
            # if you specify neither nucleus or topk,
            # then we will use the whole list
            nucleus = len(pruned_list)
            
          pruned_list = pruned_list[:nucleus]
          
          # if you want to disallow more complex tokens, you can do so here
          # for instance, if you want to disallow anything with the phrase `http`,
          # you can delete theme from the pruned_list
          # you can comment this out, I'm keeping it in for demonstration purpose
          tokens_to_disallow = []
          for _ in range(len(pruned_list)):
              if 'http' in idx2word[pruned_list[_]]:
                  tokens_to_disallow.append(_)
          pruned_list = np.delete(pruned_list, tokens_to_disallow)

          if args.topn > 0 :
            print('TOPN :: top-n alternatives:', [idx2word[_] for _ in pruned_list[:args.topn]])

          # if temperature is 0
          # just pick the first (most probable) token
          if temperature==0:
              idx = pruned_list[0]
          else:
            # else,
            # sample from the pruned_list with the logits
            chosen_idx = int(tf.random.categorical(np.expand_dims(prompt_logits[_token][pruned_list],0), num_samples=1).numpy())
            idx = pruned_list[chosen_idx]

          if args.topn > 0 :
            print('TOPN :: chosen word:', idx2word[idx])

          # assign the token for generation
          tokens_generated[0][token+1] = idx

          # clear screen if you want to
          # os.system("clear")


          tokens_generated_so_far = ' '.join([idx2word[c] for c in tokens_generated[0].squeeze()[:token+2]])
          tokens_generated_so_far = re.sub('(@@ )', '', string=tokens_generated_so_far)
          tokens_generated_so_far = re.sub('(@@ ?$)', '', string=tokens_generated_so_far)
          if not args.print_once:
            print('---------------------------------------')
            print(tokens_generated_so_far)
            print()
        print('---------------------------------------')            
        print(tokens_generated_so_far)

        #save to disk
        with open("generated_{file_number}.txt".format(file_number=file_number), "wb") as f:
            f.write(tokens_generated_so_far.encode("utf8"))

        print()

        file_number = file_number + 1
          

    except KeyboardInterrupt: #Exception as e:
        print('Continuing')



Overwriting generate2.py


In [77]:
!python2 generate2.py --model seqlen256_v1.ckpt/model.ckpt-413000.data-00000-of-00001 --print_once

W1013 04:54:38.011169 139863864698752 deprecation_wrapper.py:119] From generate2.py:7: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.

W1013 04:54:38.050618 139863864698752 deprecation_wrapper.py:119] From generate2.py:44: The name tf.random.set_random_seed is deprecated. Please use tf.compat.v1.random.set_random_seed instead.

246534 unique words
2019-10-13 04:54:38.380135: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2019-10-13 04:54:38.403572: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-10-13 04:54:38.404330: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-10-13 04:54:38.40920

In [78]:
!ls

celeb_tweets.pkl    generated_1.txt   generated_35.txt	generated_5.txt
CODE_OF_CONDUCT.md  generated_20.txt  generated_36.txt	generated_7.txt
codes		    generated_21.txt  generated_37.txt	generated_8.txt
control_codes.txt   generated_22.txt  generated_38.txt	generated_9.txt
estimator.patch     generated_23.txt  generated_39.txt	generation.py
generate2.py	    generated_24.txt  generated_3.txt	LICENSE.txt
generated_0.txt     generated_26.txt  generated_40.txt	prompts.pkl
generated_10.txt    generated_27.txt  generated_41.txt	README.md
generated_11.txt    generated_28.txt  generated_42.txt	seqlen256_v1.ckpt
generated_13.txt    generated_29.txt  generated_43.txt	source_attribution.py
generated_14.txt    generated_2.txt   generated_45.txt	transformer.py
generated_15.txt    generated_30.txt  generated_46.txt	transformer.pyc
generated_16.txt    generated_31.txt  generated_48.txt	vocab
generated_17.txt    generated_32.txt  generated_49.txt
generated_18.txt    generated_33.txt  generated_4.txt


In [79]:
!zip geneterated.zip generated_*.txt

  adding: generated_0.txt (deflated 38%)
  adding: generated_10.txt (deflated 42%)
  adding: generated_11.txt (deflated 43%)
  adding: generated_13.txt (deflated 45%)
  adding: generated_14.txt (deflated 41%)
  adding: generated_15.txt (deflated 46%)
  adding: generated_16.txt (deflated 46%)
  adding: generated_17.txt (deflated 45%)
  adding: generated_18.txt (deflated 46%)
  adding: generated_19.txt (deflated 42%)
  adding: generated_1.txt (deflated 44%)
  adding: generated_20.txt (deflated 53%)
  adding: generated_21.txt (deflated 54%)
  adding: generated_22.txt (deflated 41%)
  adding: generated_23.txt (deflated 45%)
  adding: generated_24.txt (deflated 45%)
  adding: generated_26.txt (deflated 41%)
  adding: generated_27.txt (deflated 45%)
  adding: generated_28.txt (deflated 44%)
  adding: generated_29.txt (deflated 42%)
  adding: generated_2.txt (deflated 43%)
  adding: generated_30.txt (deflated 44%)
  adding: generated_31.txt (deflated 48%)
  adding: generated_32.txt (deflated 

In [0]:
files.download("geneterated.zip")

In [0]:
s = u""" El sitio web oficial de la banda ha publicado un comunicado, que dice lo siguiente: 
 
 “En una entrevista con MTV News, los miembros de The Black Keys han confirmado su participación en la película ‘Estudio Loco’ y aseguran estar muy contentos por haber podido hacer parte de esta nueva aventura cinematográfica. 
 
 Los integrantes de The Black Keys, formados en San Francisco (EE.UU.), están encantados de poder colaborar a través de esta nueva producción para crear algo único e interesante como éste video.” 
 
 A pesar de no tener muchas dudas, las expectativas son altas. 
 
 Después, habrá que ver si es posible llegar al cine sin tocar ningún instrumento musical ni escuchar nada más que música cl"""

In [0]:
#with open("generated_{i}.txt".format(i=0), "wb") as f:
#    f.write(s.encode("utf8"))
